# Getting Started with NVFlare (Numpy)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NVIDIA/NVFlare/blob/main/examples/getting_started/pt/nvflare_pt_getting_started.ipynb)

NVFlare is an open-source framework that allows researchers and
data scientists to seamlessly move their machine learning and deep
learning workflows into a federated paradigm.

## Basic Concepts
At the heart of NVFlare lies the concept of collaboration through
"tasks." An FL controller assigns tasks (e.g., training on local data) to one or more FL clients, processes returned
results (e.g., model weight updates), and may assign additional
tasks based on these results and other factors (e.g., a pre-configured
number of training rounds). The clients run executors which can listen for tasks and perform the necessary computations locally, such as model training. This task-based interaction repeats
until the experiment’s objectives are met. 

We can also add data filters (for example, for [homomorphic encryption](https://www.usenix.org/conference/atc20/presentation/zhang-chengliang) or [differential privacy filters](https://arxiv.org/abs/1910.00962)) to the task data or results received or produced by the server or clients.

<img src="../../../docs/resources/nvflare_overview.svg" alt="NVIDIA FLARE Overview" width=75% height=75% />

## Setup environment

In [ ]:
!pip install -q -r nvflare~=2.5.0rc

## Federated Averaging with NVFlare
Given the flexible controller and executor concepts, it is easy to implement different computing & communication patterns with NVFlare, such as [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com). 

The controller's `run()` routine is responsible for assigning tasks and processing task results from the Executors. 

### Server Code
First, we use a simple implementation of the [FedAvg](https://proceedings.mlr.press/v54/mcmahan17a?ref=https://githubhelp.com) algorithm with NVFlare. 

```python
class FedAvg(BaseFedAvg):
    def run(self) -> None:
        self.info("Start FedAvg.")

        model = self.load_model()
        model.start_round = self.start_round
        model.total_rounds = self.num_rounds

        for self.current_round in range(self.start_round, self.start_round + self.num_rounds):
            self.info(f"Round {self.current_round} started.")
            model.current_round = self.current_round

            clients = self.sample_clients(self.num_clients)

            results = self.send_model_and_wait(targets=clients, data=model)

            aggregate_results = self.aggregate(results)

            model = self.update_model(model, aggregate_results)

            self.save_model(model)

        self.info("Finished FedAvg.")
```

### Client Code 
For this numpy example, we just have mock training by adding one to the data, so there are no deep learning concepts and we can focus on the flow of NVFlare.

On the client side, the training workflow is as follows:
1. Receive the model from the FL server (for this example we initialize the model in the client code to the numpy array [[1, 2, 3], [4, 5, 6], [7, 8, 9]] if the model params are empty).
2. Perform local training on the received global model
and/or evaluate the received global model for model
selection.
3. Send the new model back to the FL server.

Using NVFlare's client API, we can easily adapt machine learning code that was written for centralized training and apply it in a federated scenario.
For a general use case, there are three essential methods to achieve this using the Client API :
- `init()`: Initializes NVFlare Client API environment.
- `receive()`: Receives model from the FL server.
- `send()`: Sends the model to the FL server.

With these simple methods, the developers can use the Client API
to change their centralized training code to an FL scenario with
five lines of code changes as shown below.

```python
    import nvflare.client as flare
    
    flare.init() # 1. Initializes NVFlare Client API environment.
    input_model = flare.receive() # 2. Receives model from the FL server.
    params = input_model.params # 3. Obtain the required information from the received model.
    
    # original local training code
    new_params = local_train(params)
    
    output_model = flare.FLModel(params=new_params) # 4. Put the results in a new `FLModel`
    flare.send(output_model) # 5. Sends the model to the FL server.  
```

The full client training script is saved in a separate file, e.g. [./src/hello-numpy_fl.py](./src/hello-numpy_fl.py).

## Run an NVFlare Job
Now that we have defined the FedAvg controller to run our workflow on the FL server and our client training script to receive the global models, run local training, and send the results back to the FL server, we can put everything together using NVFlare's Job API.

#### 1. Define a FedJob
The `FedJob` is used to define how controllers and executors are placed within a federated job using the `to(object, target)` routine.

In [ ]:
from nvflare import FedJob
from nvflare.app_common.workflows.fedavg import FedAvg
from nvflare.job_config.script_runner import FrameworkType, ScriptRunner


job = FedJob(name="hello-fedavg-numpy")

#### 2. Define the Controller Workflow
Define the controller workflow with the persistor and send to server.

In [ ]:
from nvflare.app_common.np.np_model_persistor import NPModelPersistor

n_clients = 2
num_rounds = 3

persistor_id = job.to_server(NPModelPersistor(), "persistor")

controller = FedAvg(
    num_clients=n_clients,
    num_rounds=num_rounds,
    persistor_id=persistor_id,
)
job.to_server(controller)

#### 3. Add ModelSelector
Add IntimeModelSelector for global best model selection.

In [ ]:
from nvflare.app_common.widgets.intime_model_selector import IntimeModelSelector

job.to(IntimeModelSelector(key_metric="accuracy"), "server")

#### 3. Add Clients
Next, we can use the `ScriptRunner` and send it to each of the clients to run our training script.

Note that our script could have additional input arguments, such as batch size or data path, but we don't use them here for simplicity.
We can also specify, which GPU should be used to run this client, which is helpful for simulated environments.

In [ ]:
train_script = "src/hello-numpy_fl.py"

for i in range(n_clients):
    executor = ScriptRunner(
        script=train_script, script_args="", framework=FrameworkType.NUMPY
    )
    job.to(executor, f"site-{i}")

That's it!

#### 4. Optionally export the job
Now, we could export the job and submit it to a real NVFlare deployment using the [Admin client](https://nvflare.readthedocs.io/en/main/real_world_fl/operation.html) or [FLARE API](https://nvflare.readthedocs.io/en/main/real_world_fl/flare_api.html). 

In [ ]:
job.export_job("/tmp/nvflare/jobs/job_config")

#### 5. Run FL Simulation
Finally, we can run our FedJob in simulation using NVFlare's [simulator](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/fl_simulator.html) under the hood. The results will be saved in the specified `workdir`.

In [ ]:
job.simulator_run("/tmp/nvflare/jobs/workdir")